# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("real_estate_sales_data.txt",encoding="utf-8") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

Created a chunk of size 113, which is longer than the specified 100
Created a chunk of size 112, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 123, which is longer than the specified 100
Created a chunk of size 114, which is longer than the specified 100
Created a chunk of size 118, which is longer than the specified 100
Created a chunk of size 117, which is longer than the specified 100
Created a chunk of size 129, which is longer than the specified 100
Created a chunk of size 125, which is longer than the specified 100
Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 135, which is longer than the specified 100
Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 102, which is longer tha

In [5]:
docs[0]

Document(page_content='[客户问题] 宝马的保养成本是不是很高？ \n[销售回答] 宝马确实提供了高品质的服务，而且我们有多种保养计划可以根据您的需求量身定制，确保您的车辆保持最佳性能，同时控制长期成本。')

In [6]:
len(docs)

28

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

c:\Users\taosh\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "我需要一辆能应对恶劣天气的车"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 我需要一辆能应对恶劣天气的车。  
[销售回答] 宝马的xDrive四轮驱动系统能够在恶劣天气条件下提供出色的抓地力和稳定性，无论是雨、雪还是泥泞，都能让您有信心驾驶。

[客户问题] 我需要一辆有足夜储物空间的车，宝马能满足我的需求吗？
[销售回答] 绝对可以。我们的宝马 SUV 系列，比如 X3 或 X5，提供了宽敞的后备箱空间，以及灵活的座椅折叠方案，可以轻松适应不同的储物需求。

[客户问题] 我想要一辆运动型轿车，但是也得实用。  
[销售回答] 宝马的车型完美融合了运动性能与日常实用性，比如这款宝马3系，它不仅操控灵敏，加速迅猛，同时提供舒适的乘坐空间和充足的储物空间。

[客户问题] 新车太贵了，我买不起。  
[销售回答] 我们有多种金融方案可以帮助您将购车成本分散到几年内，此外，我们也有认证的二手车选择，它们经过严格检查，性能可靠，而且更加经济。



In [11]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DB5B353AF0>, search_kwargs={'k': 3})

In [14]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

c:\Users\taosh\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 我需要一辆能应对恶劣天气的车。  
[销售回答] 宝马的xDrive四轮驱动系统能够在恶劣天气条件下提供出色的抓地力和稳定性，无论是雨、雪还是泥泞，都能让您有信心驾驶。

[客户问题] 我需要一辆有足夜储物空间的车，宝马能满足我的需求吗？
[销售回答] 绝对可以。我们的宝马 SUV 系列，比如 X3 或 X5，提供了宽敞的后备箱空间，以及灵活的座椅折叠方案，可以轻松适应不同的储物需求。

[客户问题] 我想要一辆运动型轿车，但是也得实用。  
[销售回答] 宝马的车型完美融合了运动性能与日常实用性，比如这款宝马3系，它不仅操控灵敏，加速迅猛，同时提供舒适的乘坐空间和充足的储物空间。



In [15]:
docs = topK_retriever.get_relevant_documents("新车太贵了，我买不起。")

In [16]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 新车太贵了，我买不起。  
[销售回答] 我们有多种金融方案可以帮助您将购车成本分散到几年内，此外，我们也有认证的二手车选择，它们经过严格检查，性能可靠，而且更加经济。

[客户问题] 我不确定这款车是否适合我的家庭。 
[销售回答] 我们理解家庭用车的需求，这款车型不仅提供宽敞的乘坐空间，还有多项安全特性，确保您全家的舒适与安全。让我们一起看看后座空间，您一定会对它的实用性印象深刻。

[客户问题] 我怎么知道这款车真的适合我？  
[销售回答] 我们提供试驾服务，您可以亲自体验这款车的驾驶感受。此外，我们的销售顾问会根据您的生活方式和需求，提供专业建议，确保您选择最适合您的车型。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [18]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我需要一辆能应对恶劣天气的车。  
[销售回答] 宝马的xDrive四轮驱动系统能够在恶劣天气条件下提供出色的抓地力和稳定性，无论是雨、雪还是泥泞，都能让您有信心驾驶。



### 提取向量数据库中的`销售回答`

In [19]:
docs = retriever.get_relevant_documents(query)

In [20]:
docs[0].page_content

'[客户问题] 我需要一辆能应对恶劣天气的车。  \n[销售回答] 宝马的xDrive四轮驱动系统能够在恶劣天气条件下提供出色的抓地力和稳定性，无论是雨、雪还是泥泞，都能让您有信心驾驶。'

In [21]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我需要一辆能应对恶劣天气的车。  \n',
 '宝马的xDrive四轮驱动系统能够在恶劣天气条件下提供出色的抓地力和稳定性，无论是雨、雪还是泥泞，都能让您有信心驾驶。']

In [22]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [23]:
ans

'宝马的xDrive四轮驱动系统能够在恶劣天气条件下提供出色的抓地力和稳定性，无论是雨、雪还是泥泞，都能让您有信心驾驶。'

#### 尝试各种问题

In [24]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [25]:
query = "我担心电动车的充电问题"

print(sales(query))

[]


c:\Users\taosh\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [26]:
print(sales(query, 0.75))

[]


c:\Users\taosh\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


In [27]:
query = "价格200万以内的车"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

c:\Users\taosh\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



c:\Users\taosh\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['我们有多种金融方案可以帮助您将购车成本分散到几年内，此外，我们也有认证的二手车选择，它们经过严格检查，性能可靠，而且更加经济。', '我们理解家庭用车的需求，这款车型不仅提供宽敞的乘坐空间，还有多项安全特性，确保您全家的舒适与安全。让我们一起看看后座空间，您一定会对它的实用性印象深刻。', '宝马提供多种尺寸的车型，适应不同的城市生活需求。我们的一些车型特别适合城市驾驶，拥有紧凑的车身尺寸和卓越的操控性，同时配备有泊车辅助系统，使停车更加轻松。', '宝马确实提供了高品质的服务，而且我们有多种保养计划可以根据您的需求量身定制，确保您的车辆保持最佳性能，同时控制长期成本。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [28]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [39]:
qa_chain({"query": "你们有200万的车吗？"})

c:\Users\taosh\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '你们有200万的车吗？',
 'result': '对不起，作为一个人工智能，我无法销售或提供实物商品，包括汽车。我主要用于提供信息和回答问题。如果你有关于汽车的任何问题，我会尽力帮助你。'}

In [38]:
qa_chain({"query": "车内吵不吵"})

c:\Users\taosh\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '车内吵不吵',
 'result': '对不起，您的问题没有提供足够的上下文，我无法回答。您可以提供更多的信息吗？例如，您是在询问特定的车型，或者是在特定的情况下车内的噪音水平吗？'}

In [31]:
print(sales("吵不吵"))

[]


c:\Users\taosh\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [33]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(),allow_dangerous_deserialization=True)

In [34]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

c:\Users\taosh\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [35]:
qa_chain({"query": "我想买别墅，你们有么"})

c:\Users\taosh\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买别墅，你们有么', 'result': '对不起，作为一个人工智能，我无法提供购买别墅的服务。'}

In [36]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [37]:
qa_chain({"query": "我想买别墅，你们有么"})

c:\Users\taosh\AppData\Local\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能提供这样的服务。我是一个人工智能，无法帮助您购买别墅。'}

In [ ]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [ ]:
result = qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [ ]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮你购买别墅。我是一个AI助手，我主要用来提供信息和回答问题。',
 'source_documents': []}